In [41]:
import os
import time
import datetime
import pytz
import json

import boto3
import base64

In [72]:
# get the timestamp for the build
format = "%Y%m%d%H%M%S"
timestamp = time.strftime(format)

print("time stamp is: {}".format(timestamp))

# DB settings
schema_name = 'bioindex-' + timestamp

# s3 settings
s3_bucket_name = 'dig-bio-index-' + timestamp
s3_bucket_bio_index = 'dig-bio-index'

# git settings
code_directory = '/Users/mduby/BioIndex/'
git_directory = code_directory + 'bioindex_' + timestamp
git_clone_command = "git clone git@github.com:broadinstitute/dig-bioindex.git " + git_directory

# secrets settings
secret_name = "bioindex-dev"
region_name = "us-east-1"

# log
print("got schema name: {} and s3 index: {} and git directory {}".format(schema_name, s3_bucket_name, git_directory))
print("the git clone command is: {}".format(git_clone_command))


time stamp is: 20200421151455
got schema name: bioindex-20200421151455 and s3 index: dig-bio-index-20200421151455 and git directory /Users/mduby/BioIndex/bioindex_20200421151455
the git clone command is: git clone git@github.com:broadinstitute/dig-bioindex.git /Users/mduby/BioIndex/bioindex_20200421151455


In [66]:
# log
print("Running git clone command: {}".format(git_clone_command))

# clone the bioindex
os.chdir(code_directory)
clone_output = os.system(git_clone_command)

# log
print("the git process exited with code {}".format(clone_output))

Running git clone command: git clone git@github.com:broadinstitute/dig-bioindex.git /Users/mduby/BioIndex/bioindex_20200421135919
the git process exited with code 0


In [69]:
# list buckets
s3client = boto3.client('s3')

# print all the bucket names
list_buckets_resp = s3client.list_buckets()
for bucket in list_buckets_resp['Buckets']:
    if 'index' in bucket['Name']:
        print("existing bucket name: {}".format(bucket['Name']))



existing bucket name: dig-bio-index
existing bucket name: dig-bio-index-20200421132912
existing bucket name: dig-bio-index-dev


In [70]:
# create the s3 bucket
s3client.create_bucket(Bucket=s3_bucket_name)

list_buckets_resp = s3client.list_buckets()
for bucket in list_buckets_resp['Buckets']:
    if bucket['Name'] == s3_bucket_name:
        print('(Just created) --> {} - there since {}'.format(bucket['Name'], bucket['CreationDate']))

(Just created) --> dig-bio-index-20200421135919 - there since 2020-04-21 19:11:36+00:00


In [71]:
# print all the bucket names
list_buckets_resp = s3client.list_buckets()
for bucket in list_buckets_resp['Buckets']:
    if 'index' in bucket['Name']:
        print("existing bucket name after addition: {}".format(bucket['Name']))


existing bucket name after addition: dig-bio-index
existing bucket name after addition: dig-bio-index-20200421132912
existing bucket name after addition: dig-bio-index-20200421135919
existing bucket name after addition: dig-bio-index-dev


In [ ]:
# clone the bucket


In [42]:
# Use this code snippet in your app.
# If you need more information about configurations or implementing the sample code, visit the AWS docs:   
# https://aws.amazon.com/developers/getting-started/python/

import boto3
import base64
from botocore.exceptions import ClientError


def get_secret():

    secret_name = "bioindex-dev"
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    # In this sample we only handle the specific exceptions for the 'GetSecretValue' API.
    # See https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
    # We rethrow the exception by default.

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        if e.response['Error']['Code'] == 'DecryptionFailureException':
            # Secrets Manager can't decrypt the protected secret text using the provided KMS key.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InternalServiceErrorException':
            # An error occurred on the server side.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidParameterException':
            # You provided an invalid value for a parameter.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidRequestException':
            # You provided a parameter value that is not valid for the current state of the resource.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'ResourceNotFoundException':
            # We can't find the resource that you asked for.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
    else:
        # Decrypts secret using the associated KMS CMK.
        # Depending on whether the secret is a string or binary, one of these fields will be populated.
        if 'SecretString' in get_secret_value_response:
            secret = get_secret_value_response['SecretString']
        else:
            decoded_binary_secret = base64.b64decode(get_secret_value_response['SecretBinary'])

    return json.loads(secret)

In [55]:
# create an s3 bucket
s3client.create_bucket(Bucket=s3_bucket_name, ACL='private')



{'ResponseMetadata': {'RequestId': 'CD2C6CF9C6A54EFC',
  'HostId': 'k79Y1vWPZWUkM4tKvuqS0dlkIM+9WEhUEN8f3EUvEdzCU3G+jqG6rEy/PWUj/KQt4lNN9e+a1dI=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'k79Y1vWPZWUkM4tKvuqS0dlkIM+9WEhUEN8f3EUvEdzCU3G+jqG6rEy/PWUj/KQt4lNN9e+a1dI=',
   'x-amz-request-id': 'CD2C6CF9C6A54EFC',
   'date': 'Tue, 21 Apr 2020 17:29:18 GMT',
   'location': '/dig-bio-index-20200421132912',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Location': '/dig-bio-index-20200421132912'}

In [47]:
# get the secret
bio_secret = get_secret()

print("got secret of type {}".format(type(bio_secret)))
print("the secret is: {}".format(bio_secret))

got secret of type <class 'dict'>
the secret is: {'username': 'diguser', 'password': 'type2diabetes', 'engine': 'mysql', 'host': 'dig-bio-index.cxrzznxifeib.us-east-1.rds.amazonaws.com', 'port': 3306, 'dbInstanceIdentifier': 'dig-bio-index', 'dbname': 'biodev', 's3bucket': 'dig-bio-index-2014'}


In [51]:
# update the secret
session = boto3.session.Session()
client = session.client(
    service_name='secretsmanager',
    region_name=region_name
)

updated_secret = bio_secret.update({"s3bucket": s3_bucket_name})
client.update_secret(SecretId=secret_name, SecretString=json.dumps(bio_secret))



{'ARN': 'arn:aws:secretsmanager:us-east-1:005901288866:secret:bioindex-dev-oN7jhA',
 'Name': 'bioindex-dev',
 'VersionId': '5a654c67-0ef5-42ed-b407-d010c038d0ac',
 'ResponseMetadata': {'RequestId': '779c1e36-80e2-4e21-a63b-dda77d870d15',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 21 Apr 2020 16:10:08 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '155',
   'connection': 'keep-alive',
   'x-amzn-requestid': '779c1e36-80e2-4e21-a63b-dda77d870d15'},
  'RetryAttempts': 0}}